In [335]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from imblearn.over_sampling import RandomOverSampler

from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline, make_pipeline
from imblearn.pipeline import Pipeline as imb_pipe
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix, classification_report

# Load CSV


In [336]:
train_df = pd.read_csv("../csvs/train.csv")
test_df = pd.read_csv("../csvs/test.csv")

In [337]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [338]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [339]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Feature Selection


In [340]:
train_df.groupby(train_df["Pclass"], as_index=False)["Survived"].agg(["count", "mean"])

,Pclass,count,mean
0,1,216,0.629630
1,2,184,0.472826
2,3,491,0.242363


In [341]:
train_df.groupby(train_df["Sex"], as_index=False)["Survived"].agg(["count", "mean"])

,Sex,count,mean
0,female,314,0.742038
1,male,577,0.188908


In [342]:
age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
age_cut = pd.cut(train_df["Age"], bins=age_bins)
train_df.groupby(age_cut, as_index=False)["Survived"].agg(["count", "mean"])

,Age,count,mean
0,"(0, 10]",64,0.593750
1,"(10, 20]",115,0.382609
2,"(20, 30]",230,0.365217
3,"(30, 40]",155,0.445161
4,"(40, 50]",86,0.383721
5,"(50, 60]",42,0.404762
6,"(60, 70]",17,0.235294
7,"(70, 80]",5,0.200000


In [343]:
train_df.groupby(train_df["SibSp"], as_index=False)["Survived"].agg(["count", "mean"])

,SibSp,count,mean
0,0,608,0.345395
1,1,209,0.535885
2,2,28,0.464286
3,3,16,0.250000
4,4,18,0.166667
5,5,5,0.000000
6,8,7,0.000000


In [344]:
train_df.groupby(train_df["Parch"], as_index=False)["Survived"].agg(["count", "mean"])

,Parch,count,mean
0,0,678,0.343658
1,1,118,0.550847
2,2,80,0.500000
3,3,5,0.600000
4,4,4,0.000000
5,5,5,0.200000
6,6,1,0.000000


In [345]:
train_df["FamilySize"] = train_df["SibSp"] + train_df["Parch"] + 1
test_df["FamilySize"] = test_df["SibSp"] + train_df["Parch"] + 1

In [346]:
train_df.groupby(train_df["FamilySize"])["Survived"].agg(["count", "mean"])

,count,mean
FamilySize,,
1,537,0.303538
2,161,0.552795
3,102,0.578431
4,29,0.724138
5,15,0.200000
6,22,0.136364
7,12,0.333333
8,6,0.000000
11,7,0.000000


In [347]:
family_map = {
    1: "Alone",
    2: "Small",
    3: "Small",
    4: "Small",
    5: "Medium",
    6: "Medium",
    7: "Large",
    8: "Large",
    9: "Large",
    10: "Large",
    11: "Large",
}
train_df["FamilyGroupSize"] = train_df["FamilySize"].map(family_map)
test_df["FamilyGroupSize"] = test_df["FamilySize"].map(family_map)

In [348]:
fare_bins = [0, 100, 200, 300, 400, 500, 600]
fare_cut = pd.cut(train_df["Fare"], bins=fare_bins)
train_df.groupby(fare_cut, as_index=False)["Survived"].agg(["count", "mean"])

,Fare,count,mean
0,"(0, 100]",823,0.366950
1,"(100, 200]",33,0.757576
2,"(200, 300]",17,0.647059
3,"(300, 400]",0,NaN
4,"(400, 500]",0,NaN
5,"(500, 600]",3,1.000000


In [349]:
train_df.groupby(["Embarked"], as_index=False)["Survived"].agg(["count", "mean"])

,Embarked,count,mean
0,C,168,0.553571
1,Q,77,0.389610
2,S,644,0.336957


In [350]:
train_df["name_title"] = (
    train_df["Name"]
    .str.split(",", expand=True)[1]
    .str.split(".", expand=True)[0]
    .apply(lambda x: x.strip())
)
test_df["name_title"] = (
    test_df["Name"]
    .str.split(",", expand=True)[1]
    .str.split(".", expand=True)[0]
    .apply(lambda x: x.strip())
)

In [351]:
train_df["name_title"].value_counts()

name_title
Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Col               2
Mlle              2
Major             2
Ms                1
Mme               1
Don               1
Lady              1
Sir               1
Capt              1
the Countess      1
Jonkheer          1
Name: count, dtype: int64

In [352]:
name_map = {
    "Dr": "Professional",
    "Rev": "Professional",
    "Col": "Military",
    "Mlle": "Miss",
    "Major": "Military",
    "Ms": "Miss",
    "Mme": "Mrs",
    "Don": "Nobility",
    "Lady": "Nobility",
    "Sir": "Nobility",
    "Capt": "Military",
    "the Countess": "Nobility",
    "Jonkheer": "Nobility",
}
train_df["name_title"] = train_df["name_title"].replace(name_map)
test_df["name_title"] = test_df["name_title"].replace(name_map)

In [353]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,FamilyGroupSize,name_title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,Small,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,Small,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Alone,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,Small,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,Alone,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1,Alone,Professional
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,Alone,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,4,Small,Miss
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,Alone,Mr


In [354]:
train_df["TicketNumber"] = train_df["Ticket"].apply(lambda x: x.split()[-1])
test_df["TicketNumber"] = test_df["Ticket"].apply(lambda x: x.split()[-1])
train_df["TicketNumber"]

0        21171
1        17599
2      3101282
3       113803
4       373450
        ...   
886     211536
887     112053
888       6607
889     111369
890     370376
Name: TicketNumber, Length: 891, dtype: object

In [355]:
train_df["TicketNumberCounts"] = train_df.groupby(["TicketNumber"])[
    "TicketNumber"
].transform("count")
test_df["TicketNumberCounts"] = test_df.groupby(["TicketNumber"])[
    "TicketNumber"
].transform("count")

In [356]:
train_df.groupby(["TicketNumberCounts"], as_index=False)["Survived"].agg(
    ["count", "mean"]
)

,TicketNumberCounts,count,mean
0,1,544,0.295956
1,2,188,0.569149
2,3,66,0.712121
3,4,44,0.500000
4,5,10,0.000000
5,6,18,0.000000
6,7,21,0.238095


In [357]:
train_df["Ticket_Prefix"] = np.where(
    train_df["Ticket"].str.split(" ", n=1, expand=True)[1].notna(),
    train_df["Ticket"].str.split(" ", n=1, expand=True)[0].apply(lambda x: x.strip()),
    "Blank",
)
test_df["Ticket_Prefix"] = np.where(
    test_df["Ticket"].str.split(" ", n=1, expand=True)[1].notna(),
    test_df["Ticket"].str.split(" ", n=1, expand=True)[0].apply(lambda x: x.split()),
    "Blank",
)

In [358]:
train_df["Ticket_Prefix"].value_counts()

Ticket_Prefix
Blank         665
PC             60
C.A.           27
STON/O         12
A/5            10
W./C.           9
CA.             8
SOTON/O.Q.      8
A/5.            7
SOTON/OQ        7
STON/O2.        6
CA              6
C               5
S.O.C.          5
SC/PARIS        5
F.C.C.          5
SC/Paris        4
A/4.            3
PP              3
A/4             3
S.O./P.P.       3
SC/AH           3
A./5.           2
P/PP            2
A.5.            2
WE/P            2
SOTON/O2        2
S.C./PARIS      2
S.C./A.4.       1
Fa              1
S.O.P.          1
SO/C            1
S.P.            1
A4.             1
W.E.P.          1
A/S             1
SC              1
SW/PP           1
SCO/W           1
W/C             1
S.W./PP         1
F.C.            1
C.A./SOTON      1
Name: count, dtype: int64

In [359]:
train_df["TicketLocation"] = train_df["Ticket_Prefix"].replace(
    {
        "SOTON/O.Q": "SOTON/OQ",
        "C.A": "CA",
        "CA.": "CA",
        "S.C./PARIS": "SC/Paris",
        "S.C./PARIS": "SC/Paris",
        "A/4": "A/4",
        "A/5": "A/5",
        "A/5.": "A/5",
        "A./5": "A/5",
        "W./C.": "W/C",
    }
)
test_df["TicketLocation"] = test_df["Ticket_Prefix"].replace(
    {
        "SOTON/O.Q": "SOTON/OQ",
        "C.A": "CA",
        "CA.": "CA",
        "S.C./PARIS": "SC/Paris",
        "S.C./PARIS": "SC/Paris",
        "A/4": "A/4",
        "A/5": "A/5",
        "A/5.": "A/5",
        "A./5": "A/5",
        "W./C.": "W/C",
    }
)

In [360]:
train_df["TicketLocation"].value_counts()

TicketLocation
Blank         665
PC             60
C.A.           27
A/5            17
CA             14
STON/O         12
W/C            10
SOTON/O.Q.      8
SOTON/OQ        7
STON/O2.        6
SC/Paris        6
SC/PARIS        5
F.C.C.          5
C               5
S.O.C.          5
A/4             3
PP              3
SC/AH           3
A/4.            3
S.O./P.P.       3
A./5.           2
SOTON/O2        2
WE/P            2
P/PP            2
A.5.            2
S.O.P.          1
A4.             1
S.P.            1
S.C./A.4.       1
W.E.P.          1
SO/C            1
SC              1
SCO/W           1
Fa              1
SW/PP           1
S.W./PP         1
A/S             1
F.C.            1
C.A./SOTON      1
Name: count, dtype: int64

In [361]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,FamilyGroupSize,name_title,TicketNumber,TicketNumberCounts,Ticket_Prefix,TicketLocation
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,Small,Mr,21171,1,A/5,A/5
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,Small,Mrs,17599,1,PC,PC
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Alone,Miss,3101282,1,STON/O2.,STON/O2.
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,Small,Mrs,113803,2,Blank,Blank
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,Alone,Mr,373450,1,Blank,Blank
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1,Alone,Professional,211536,1,Blank,Blank
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,Alone,Miss,112053,1,Blank,Blank
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,4,Small,Miss,6607,2,W./C.,W/C
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,Alone,Mr,111369,1,Blank,Blank


In [362]:
train_df["Cabin"]

0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: Cabin, Length: 891, dtype: object

In [363]:
train_df["Cabin"].fillna("U", inplace=True)
test_df["Cabin"].fillna("U", inplace=True)

In [364]:
split_cabin = train_df["Cabin"].str.split(" ", expand=True)[0]
split_cabin_test = test_df["Cabin"].str.split(" ", expand=True)[0]
split_cabin.str[0].value_counts()

0
U    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: count, dtype: int64

In [365]:
train_df["Cabin"] = split_cabin.str[0]
test_df["Cabin"] = split_cabin_test.str[0]

In [366]:
train_df.groupby(["Cabin"])["Survived"].agg(["count", "mean"])

,count,mean
Cabin,,
A,15,0.466667
B,47,0.744681
C,59,0.593220
D,33,0.757576
E,32,0.750000
F,13,0.615385
G,4,0.500000
T,1,0.000000
U,687,0.299854


In [367]:
train_df["CabinAssigned"] = train_df["Cabin"].apply(lambda x: 0 if x in ["U"] else 1)
test_df["CabinAssigned"] = test_df["Cabin"].apply(lambda x: 0 if x in ["U"] else 1)

In [368]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,FamilyGroupSize,name_title,TicketNumber,TicketNumberCounts,Ticket_Prefix,TicketLocation,CabinAssigned
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S,2,Small,Mr,21171,1,A/5,A/5,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C,2,Small,Mrs,17599,1,PC,PC,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U,S,1,Alone,Miss,3101282,1,STON/O2.,STON/O2.,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C,S,2,Small,Mrs,113803,2,Blank,Blank,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U,S,1,Alone,Mr,373450,1,Blank,Blank,0


# Pre-processing


In [369]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PassengerId         891 non-null    int64  
 1   Survived            891 non-null    int64  
 2   Pclass              891 non-null    int64  
 3   Name                891 non-null    object 
 4   Sex                 891 non-null    object 
 5   Age                 714 non-null    float64
 6   SibSp               891 non-null    int64  
 7   Parch               891 non-null    int64  
 8   Ticket              891 non-null    object 
 9   Fare                891 non-null    float64
 10  Cabin               891 non-null    object 
 11  Embarked            889 non-null    object 
 12  FamilySize          891 non-null    int64  
 13  FamilyGroupSize     891 non-null    object 
 14  name_title          891 non-null    object 
 15  TicketNumber        891 non-null    object 
 16  TicketNu

In [370]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PassengerId         418 non-null    int64  
 1   Pclass              418 non-null    int64  
 2   Name                418 non-null    object 
 3   Sex                 418 non-null    object 
 4   Age                 332 non-null    float64
 5   SibSp               418 non-null    int64  
 6   Parch               418 non-null    int64  
 7   Ticket              418 non-null    object 
 8   Fare                417 non-null    float64
 9   Cabin               418 non-null    object 
 10  Embarked            418 non-null    object 
 11  FamilySize          418 non-null    float64
 12  FamilyGroupSize     417 non-null    object 
 13  name_title          418 non-null    object 
 14  TicketNumber        418 non-null    object 
 15  TicketNumberCounts  418 non-null    int64  
 16  Ticket_P

In [415]:
train_df["Age"].fillna(train_df["Age"].mean(), inplace=True)
# train_df["Embarked"] has empty rows
test_df["Age"].fillna(test_df["Age"].mean(), inplace=True)
test_df["Fare"].fillna(test_df["Fare"].mean(), inplace=True)
# test_df['FamilyGroupSize'] has an empty row

In [372]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,FamilyGroupSize,name_title,TicketNumber,TicketNumberCounts,Ticket_Prefix,TicketLocation,CabinAssigned
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S,2,Small,Mr,21171,1,A/5,A/5,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C,2,Small,Mrs,17599,1,PC,PC,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U,S,1,Alone,Miss,3101282,1,STON/O2.,STON/O2.,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C,S,2,Small,Mrs,113803,2,Blank,Blank,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U,S,1,Alone,Mr,373450,1,Blank,Blank,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,U,S,1,Alone,Professional,211536,1,Blank,Blank,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B,S,1,Alone,Miss,112053,1,Blank,Blank,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,U,S,4,Small,Miss,6607,2,W./C.,W/C,0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,C,1,Alone,Mr,111369,1,Blank,Blank,1


In [416]:
X = train_df.drop(["Survived"], axis=1)
y = train_df["Survived"]

In [417]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [418]:
ohe_cols = ["Sex", "Cabin", "name_title", "Embarked"]
ode_cols = ["FamilyGroupSize"]

In [419]:
ohe = OneHotEncoder(sparse_output=False)
ode = OrdinalEncoder()
SI = SimpleImputer(strategy="most_frequent")

## Pipelines


In [420]:
ohe_pipeline = Pipeline(
    steps=[
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
    ]
)

In [421]:
ode_pipeline = Pipeline(
    steps=[
        ("impute", SimpleImputer(strategy="most_frequent")),
        (
            "ode",
            OrdinalEncoder(
                handle_unknown="use_encoded_value", unknown_value=np.nan, dtype="float"
            ),
        ),
    ]
)

In [422]:
column_pipeline = ColumnTransformer(
    [
        ("ode_pipeline", ode_pipeline, ["FamilyGroupSize"]),
        ("ohe_pipeline", ohe_pipeline, ["Sex", "Cabin", "name_title", "Embarked"]),
        (
            "passthrough",
            "passthrough",
            [
                "Pclass",
                "Age",
                "Fare",
                "FamilySize",
                "TicketNumberCounts",
                "CabinAssigned",
            ],
        ),
    ],
    remainder="drop",
    n_jobs=-1
)

# Models

## RandomForest

In [380]:
# rfc_pipeline = imb_pipe([
#     # ('scaler', StandardScaler()), - Scaling is unnecessary for RandomForest
#     #('sampler', RandomOverSampler(random_state=42)),
#     ('rfc', RandomForestClassifier())
# ])
rfc_pipeline = RandomForestClassifier()

In [381]:
param_grid = {
    'n_estimators': [100, 150, 200],
    'min_samples_split': [5, 10, 15],
    'max_depth': [8, 9, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy', 'log_loss']
}

In [382]:
cv = StratifiedKFold(n_splits=5)
CV_rfc = GridSearchCV(estimator=rfc_pipeline, param_grid=param_grid, cv=cv)

In [383]:
final_rfc_pipeline = make_pipeline(column_pipeline, CV_rfc)
final_rfc_pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('gridsearchcv', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('ode_pipeline', ...), ('ohe_pipeline', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the outp

In [384]:
y_pred = final_rfc_pipeline.predict(X_test)

In [385]:
print(f"Best Estimator: {CV_rfc.best_estimator_}")
print(f"Best Parameters: {CV_rfc.best_params_}")
print(f"Best Score: {CV_rfc.best_score_}")

Best Estimator: RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=2,
                       min_samples_split=5)
Best Parameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best Score: 0.8458085512288184


In [386]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[121  16]
 [ 27  59]]
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       137
           1       0.79      0.69      0.73        86

    accuracy                           0.81       223
   macro avg       0.80      0.78      0.79       223
weighted avg       0.81      0.81      0.80       223



## Decision Trees

In [390]:
dtc = DecisionTreeClassifier()

In [401]:
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'min_samples_split': [2, 5, 7, 9],
    'min_samples_leaf': [1, 3, 5, 9, 11],
    'max_depth': [10, 20, 70, 100]
}

In [402]:
cv = StratifiedKFold(n_splits=5)
dtc_cv = GridSearchCV(estimator=dtc, param_grid=param_grid, cv=cv)

In [403]:
final_dtc_cv = make_pipeline(column_pipeline, dtc_cv)
final_dtc_cv.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('gridsearchcv', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('ode_pipeline', ...), ('ohe_pipeline', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the outp

In [404]:
y_pred = final_dtc_cv.predict(X_test)

In [411]:
print(f"Best Estimator: {dtc_cv.best_estimator_}")
print(f"Best Parameters: {dtc_cv.best_params_}")
print(f"Best Score: {dtc_cv.best_score_}")

Best Estimator: DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_split=5)
Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best Score: 0.8233307148468185


In [412]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[119  18]
 [ 34  52]]
              precision    recall  f1-score   support

           0       0.78      0.87      0.82       137
           1       0.74      0.60      0.67        86

    accuracy                           0.77       223
   macro avg       0.76      0.74      0.74       223
weighted avg       0.76      0.77      0.76       223



## KNN

In [430]:
knn_pipeline = imb_pipe([
    ('scaler', StandardScaler()),
    ('sampler', RandomOverSampler(random_state=42)),
    ('knn', KNeighborsClassifier())
])

In [431]:
param_grid = {
    'knn__n_neighbors': [5, 7, 9, 11, 15],
    'knn__weights': ['uniform', 'distance'],
    'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'knn__n_jobs': [-1, 1]
}

In [432]:
cv = StratifiedKFold(n_splits=5)
knn_cv = GridSearchCV(knn_pipeline, param_grid=param_grid, cv=cv)

In [433]:
final_knn_cv = make_pipeline(column_pipeline, knn_cv)
final_knn_cv.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('gridsearchcv', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('ode_pipeline', ...), ('ohe_pipeline', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the outp

In [435]:
y_pred = final_knn_cv.predict(X_test)

In [436]:
print(f"Best Estimator: {knn_cv.best_estimator_}")
print(f"Best Parameters: {knn_cv.best_params_}")
print(f"Best Score: {knn_cv.best_score_}")

Best Estimator: Pipeline(steps=[('scaler', StandardScaler()),
                ('sampler', RandomOverSampler(random_state=42)),
                ('knn', KNeighborsClassifier(n_jobs=-1, n_neighbors=7))])
Best Parameters: {'knn__algorithm': 'auto', 'knn__n_jobs': -1, 'knn__n_neighbors': 7, 'knn__weights': 'uniform'}
Best Score: 0.8023903041185052


In [437]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[115  22]
 [ 22  64]]
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       137
           1       0.74      0.74      0.74        86

    accuracy                           0.80       223
   macro avg       0.79      0.79      0.79       223
weighted avg       0.80      0.80      0.80       223

